In [48]:
import os
import pandas as pd
from datetime import datetime


In [34]:
input_file = "/Users/usi/Desktop/Adversarial_XSS/output/all.csv"
output_file = "/Users/usi/Desktop/Adversarial_XSS/output/filtered.csv"

In [35]:
df = pd.read_csv(input_file)
df.head()

,Unnamed: 0,scopus_id,title,publication_name,issn,isbn,eissn,volume,page_range,cover_date,doi,citation_count,affiliation,aggregation_type,subtype_description,authors,full_text
0,0,85167516396,An LSTM based cross-site scripting attack dete...,Journal of Cloud Computing,NaN,NaN,2192113X,12,NaN,2023-12-01,10.1186/s13677-023-00483-x,0,[{'name': 'Beijing University of Posts and Tel...,Journal,Article,"['57222408825', '58529910200', '57222400611', ...",NaN
1,1,85168730096,ZTWeb: Cross site scripting detection based on...,Computers and Security,01674048,NaN,NaN,134,NaN,2023-11-01,10.1016/j.cose.2023.103434,0,"[{'name': 'Tianjin University', 'city': 'Tianj...",Journal,Article,"['58548008400', '56984876600', '57022407900', ...",https://api.elsevier.com/content/article/eid/1...
2,2,85150489378,Detection of cross-site scripting (XSS) attack...,Artificial Intelligence Review,02692821,NaN,15737462,56,12725-12769,2023-11-01,10.1007/s10462-023-10433-3,2,"[{'name': 'Chandigarh University', 'city': 'Mo...",Journal,Article,"['57212501970', '57210605091', '57197866300']",NaN
3,3,85163528486,Cost-effective detection system of cross-site ...,Results in Engineering,NaN,NaN,25901230,19,NaN,2023-09-01,10.1016/j.rineng.2023.101266,0,"[{'name': 'Princess Sumaya University', 'city'...",Journal,Article,['57939384700'],https://api.elsevier.com/content/article/eid/1...
4,4,85169151006,State-Sensitive Black-Box Web Application Scan...,Applied Sciences (Switzerland),NaN,NaN,20763417,13,NaN,2023-08-01,10.3390/app13169212,0,[{'name': 'National University of Defense Tech...,Journal,Article,"['58555125300', '55738311000', '57190569944', ...",NaN


In [36]:
len(df)

487

In [37]:
df = df.drop_duplicates(subset=["title"])
len(df)

482

In [38]:
df.aggregation_type.value_counts()

aggregation_type
Conference Proceeding    238
Journal                  159
Book Series               79
Book                       6
Name: count, dtype: int64

In [39]:
df = df.query("aggregation_type == 'Conference Proceeding' | aggregation_type == 'Journal'")
len(df)

397

In [40]:
df.subtype_description.value_counts()

subtype_description
Conference Paper     231
Article              147
Conference Review     10
Review                 8
Retracted              1
Name: count, dtype: int64

In [41]:
df = df.query("subtype_description == 'Conference Paper' | subtype_description == 'Article'")
len(df)

378

In [49]:
date_format = '%Y-%m-%d'
df['date'] = df.cover_date.map(lambda x: datetime.strptime(x,date_format))

In [50]:
df['year'] = df.date.dt.year
df['year'].value_counts()

year
2022    46
2023    41
2021    41
2019    38
2020    33
2016    26
2017    24
2018    22
2013    17
2015    16
2012    16
2014    14
2011    10
2010     8
2009     7
2008     7
2007     5
2006     4
2004     2
1970     1
Name: count, dtype: int64

In [51]:
df = df.query("year >= 2020")
len(df)

161

In [52]:
df = df[["title", "year", "doi", "full_text"]]
df.to_csv(output_file, index=False)